# Exercise: Implementing Gradient Descent

Implementing gradient descent
Okay, now we know how to update our weights:

You've seen how to implement that for a single update, but how do we translate that code to calculate many weight updates so our network will learn?

As an example, I'm going to have you use gradient descent to train a network on graduate school admissions data (found at http://www.ats.ucla.edu/stat/data/binary.csv). This dataset has three input features: GRE score, GPA, and the rank of the undergraduate school (numbered 1 through 4). Institutions with rank 1 have the highest prestige, those with rank 4 have the lowest.

The goal here is to predict if a student will be admitted to a graduate program based on these features. For this, we'll use a network with one output layer with one unit. We'll use a sigmoid function for the output unit activation.

## Data Prep

### Data cleanup
You might think there will be three input units, but we actually need to transform the data first.

The rank feature is categorical, the numbers don't encode any sort of relative values.
Rank 2 is not twice as much as rank 1, rank 3 is not 1.5 more than rank 2.
Instead, we need to use dummy variables to encode rank, splitting the data into four new columns encoded with ones or zeros.
Rows with rank 1 have one in the rank 1 dummy column, and zeros in all other columns.
Rows with rank 2 have one in the rank 2 dummy column, and zeros in all other columns. Andexercise below.

### Standardizing the GRE and GPA Data
We'll also need to standardize the GRE and GPA data, which means scaling the values such that they have zero mean and a standard deviation of 1.

- This is necessary because the sigmoid function squashes really small and really large inputs.
- The gradient of really small and large inputs is zero, which means that the gradient descent step will go to zero too.
- Since the GRE and GPA values are fairly large, we have to be really careful about how we initialize the weights or the gradient descent steps will die off and the network won't train.
- Instead, if we standardize the data, we can initialize the weights easily and everyone is happy.

This is just a brief run-through, you'll learn more about preparing data later. If you're interested in how I did this, check out the data_prep.py file in the programming exercise below.

In [4]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

admissions

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [14]:
# Make dummy variables for rank
data = pd.get_dummies(admissions, prefix='rank', columns=['rank'], dtype=int)

data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


In [15]:
# Standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
...,...,...,...,...,...,...,...
395,0,0.279614,1.603135,0,1,0,0
396,0,-0.239793,-0.919418,0,0,1,0
397,0,-1.105469,-1.996759,0,1,0,0
398,0,0.972155,0.683454,0,1,0,0


In [16]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

In [17]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

features

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0
...,...,...,...,...,...,...
393,0.279614,0.946220,0,1,0,0
134,-0.239793,-1.155908,0,1,0,0
306,-0.412928,-0.577822,1,0,0,0
383,0.625884,1.603135,1,0,0,0


In [18]:
targets

209    0
280    0
33     1
210    0
93     0
      ..
393    1
134    0
306    1
383    0
319    0
Name: admit, Length: 360, dtype: int64

Now that the data is ready, we see that there are six input features: gre, gpa, and the four rank dummy variables.

## Mean Square Error
We're going to make a small change to how we calculate the error here. Instead of the SSE, we're going to use the mean of the square errors (MSE). Now that we're using a lot of data, summing up all the weight steps can lead to really large updates that make the gradient descent diverge. To compensate for this, you'd need to use a quite small learning rate. Instead, we can just divide by the number of records in our data, 
�
m to take the average. This way, no matter how much data we use, our learning rates will typically be in the range of 0.01 to 0.001. Then, we can use the MSE (shown below) to calculate the gradient and the result is the same as before, just averaged instead of summed.

## Implementing with NumPy
For the most part, this is pretty straightforward with NumPy.

First, you'll need to initialize the weights. We want these to be small such that the input to the sigmoid is in the linear region near 0 and not squashed at the high and low ends. It's also important to initialize them randomly so that they all have different starting values and diverge, breaking symmetry. So, we'll initialize the weights from a normal distribution centered at 0. A good value for the scale is 
1
/
�
1/ 
n
​
  where 
�
n is the number of input units. This keeps the input to the sigmoid low for increasing numbers of input units.

In [21]:
import numpy as np
from data_prep import features, targets, features_test, targets_test

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def update_weights(weights, features, targets, learnrate):
    """
    Complete a single epoch of gradient descent and return updated weights
    """
    del_w = np.zeros(weights.shape)
    # Loop through all records, x is the input, y is the target
    for x, y in zip(features.values, targets):
        # TODO: Calculate the output of f(h) by passing h (the dot product
        # of x and weights) into the activation function (sigmoid).
        # Replace None with appropriate code
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error by subtracting the network output
        # from the target (y).
        # Replace None with appropriate code
        error = y - output

        # TODO: Calculate the error term by multiplying the error by the
        # gradient. Recall that the gradient of the sigmoid f(h) is
        # f(h)*(1−f(h)) so you do not need to call any additional
        # functions and can simply apply this formula to the output and
        # error you already calculated.
        # Replace None with appropriate code
        error_term = error * output * (1 - output)

        # TODO: Update the weight step by multiplying the error term by
        # the input (x) and adding this to the current weight step.
        # Replace 0 with appropriate code
        del_w += error_term * x

    n_records = features.shape[0]
    # TODO: Update the weights by adding the learning rate times the
    # change in weights divided by the number of records.
    # Replace 0 with appropriate code
    weights += learnrate * del_w / n_records
    
    return weights

def gradient_descent(features, targets, epochs=1000, learnrate=0.5):
    """
    Perform the complete gradient descent process on a given dataset
    """
    # Use to same seed to make debugging easier
    np.random.seed(42)
    
    # Initialize loss and weights
    last_loss = None
    n_features = features.shape[1]
    weights = np.random.normal(scale=1/n_features**.5, size=n_features)

    # Repeatedly update the weights based on the number of epochs
    for e in range(epochs):
        weights = update_weights(weights, features, targets, learnrate)

        # Printing out the MSE on the training set every 10 epochs.
        # Initially this will print the same loss every time. When all of
        # the TODOs are complete, the MSE should decrease with each
        # printout
        if e % (epochs / 10) == 0:
            out = sigmoid(np.dot(features, weights))
            loss = np.mean((out - targets) ** 2)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            
    return weights

# Calculate accuracy on test data
weights = gradient_descent(features, targets)
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

ModuleNotFoundError: No module named 'data_prep'